In [94]:
import pandas as pd
import numpy as np
import os

In [95]:
# Đọc các master file

try:
    master_data_dir = 'D:\SGC\TPTS_Project\master_data'
    # Danh sách store Active theo MMS
    mms_active_store = pd.read_excel(master_data_dir + '\MMS_Store_Active.xlsx')

    # Bảng Location D&F
    df_loc = pd.read_csv(master_data_dir + '\DF_All_Loc.csv',encoding='iso-8859-1')

    # Danh sách store theo CBNC-PKD
    pkd_store = pd.read_excel(master_data_dir + '\Store_CBNC.xlsx')

    # Danh mục SKU theo từng nhóm CF
    dm_cf_sku = pd.read_excel(master_data_dir + '\Danh_muc_CBNC_CF.xlsx', header=1)

    # Danh sách CF theo từng nhóm
    dm_cf_store_group =pd.read_excel(master_data_dir + '\Danh_muc_CBNC_CF.xlsx', sheet_name='2. Group DS CF')
    
except:
    print('Error in reading files')


In [96]:
dm_cf_sku = dm_cf_sku[~dm_cf_sku['SKU'].isin([3571173,3571174,3322308, 3322309])]

In [97]:
# Đọc các master file cần thiết
def get_store_list(store_list = 'PKD', export_check_store=False):
    """ 
    Hàm có chức năng sử dụng Pandas để đọc các master data file có trong folder master_data, đồng thời filter theo mục đích của từng bảng.

    Input: 
        - Tùy chọn export_check_store, True nếu cần xuất file store active nhưng không có trong list PKD gửi, False nếu không cần
        - Tùy chọn danh sách muốn lấy, nếu 'PKD' thì list được return sẽ là danh sách store theo PKD, nếu 'DF' thì list được return sẽ là danh sách store theo D&F.
    Output: D
    """

    # Lọc bảng Location để chỉ lấy ID các store và còn hoạt động
    df_store = df_loc[(df_loc['HIERARCHYLEVEL']=='STORE') &
                    (df_loc['UDC_Delete'] == False)]
    
    # Lọc bảng Location 1 lần nữa theo danh sách MMS Store Active để đảm bảo loại hoàn toàn Store không còn hoạt động
    df_store_active = df_store[(df_store['*Loc'].isin(mms_active_store['Store Number'].astype('str')))]
    # df_store_active = df_store

    # Xuất file gồm các store đang active nhưng không có trong danh sách của PKD
    if export_check_store:
        df_store_active[~df_store_active['*Loc'].isin(pkd_store['Store Id'].astype('str'))].to_csv('output-files/store_to_check.csv')
    
    if store_list == 'DF':
        return df_store_active['*Loc'].astype('str').tolist()
    elif store_list == 'PKD':
        return pkd_store['Store Id'].astype('str').tolist()
    else:
        print(f"Không có danh sách store nào theo {store_list}")
        return None

In [98]:
pkd_list = get_store_list('PKD')
df_list = get_store_list('DF')
store_list = [x for x in pkd_list if x in df_list]
np.setdiff1d(pkd_list, df_list)

array(['12701', '2173', '56202', '6901101', '6902001', '799', '9110',
       '9116', '9130', '9153'], dtype='<U7')

Trong thời điểm hiện tại, vẫn sử dụng danh mục store chính là danh mục PKD gửi

In [99]:
# Đọc file order ngày hiện tại
orders = pd.read_csv('input-orders/orders_14-4-2024.csv', encoding='iso-8859-1')
orders= orders[orders['ARRIVAL_DATE'] == '4/14/24']
orders.head()

,*LOADID,*ITEM,*SOURCE,*DEST,QTY,SHIP_DATE,ARRIVAL_DATE,STT_NCC,STT_SKU,UDC_DAY,UDC_NOTE,UDC_TIME,UNITPRICE
0,8760364,3001608,10324,174,NaN,4/13/24,4/14/24,NaN,45.0,1D,NaN,11:15,"73,000.000"
1,8760364,3001614,10324,174,NaN,4/13/24,4/14/24,NaN,44.0,1D,NaN,11:15,"144,000.000"
2,8760364,3065918,10324,174,NaN,4/13/24,4/14/24,NaN,71.0,1D,NaN,11:15,"244,200.000"
3,8760364,3065920,10324,174,NaN,4/13/24,4/14/24,NaN,46.0,1D,NaN,11:15,"244,200.000"
4,8760364,3371966,10324,174,NaN,4/13/24,4/14/24,NaN,47.0,1D,NaN,11:15,"214,000.000"


Ok giờ kiểm tra xem orders:
- Có đầy đủ store chưa? Nếu thiếu thì thiếu store nào liệt kê ra
- Mỗi store có bao nhiêu SKU?

In [100]:
count_of_store = orders['*DEST'].nunique()
count_of_missing_store = len(np.setdiff1d(np.array(store_list), orders['*DEST'].unique()))
count_of_redundant_store = len(np.setdiff1d(orders['*DEST'].unique(),np.array(store_list)))


In [101]:
if count_of_missing_store == 0:
    print('No missing store')
else:
    print(np.setdiff1d(orders['*DEST'].unique(),np.array(store_list)))

No missing store


In [102]:
if count_of_redundant_store == 0:
    print('No redundant store')
else:
    print(np.setdiff1d(orders['*DEST'].unique(),np.array(store_list)))

[ 9110  9116  9130  9153 12701 56202 69011 69020]


In [103]:
count = orders.groupby(by='*DEST')['*ITEM'].count()
count.unique()

array([437, 171, 162,  66, 118, 388], dtype=int64)

In [104]:
report_df = pd.DataFrame(columns=['Store','SL SKU có trong Order','Nhóm Store','SL SKU của Nhóm Store', 'SL SKU Thiếu','SL SKU Thừa', 'SKU Thiếu', 'SKU Thừa'])


sku_list_each_group = {}
for store_group in dm_cf_store_group['NHOM DS'].unique():
    sku_list_each_group[store_group] = dm_cf_sku[~dm_cf_sku[store_group].isna()]['SKU'].tolist()
sku_list_each_group['No Group'] = pd.read_csv(master_data_dir + '\TPTS_All_Items.csv')['*Item'].tolist()


for store in orders['*DEST'].unique():
    # Get group of store
    group_of_store = dm_cf_store_group.loc[dm_cf_store_group['Mã ST']==store,'NHOM DS'].tolist()
    if not len(group_of_store) == 0:
        group_of_store = group_of_store[0]
    else:
        group_of_store = 'No Group'
    
    # Danh sách SKU của store có trong order
    sku_list_of_store = orders[orders['*DEST']==store]['*ITEM']
    # Danh sách SKU của nhóm store
    sku_list_of_group = sku_list_each_group[group_of_store]
    # Số lượng SKU có trong order mà không thuộc nhóm store này
    count_of_redundant_sku = len(np.setdiff1d(sku_list_of_store, sku_list_of_group))
    # Số lượng SKU có trong nhóm store nhưng không có trong order
    count_of_missing_sku = len(np.setdiff1d(sku_list_of_group, sku_list_of_store))

    report_df = pd.concat([report_df, pd.DataFrame([{'Store': store,
                     'SL SKU có trong Order': len(sku_list_of_store),
                     'Nhóm Store': group_of_store,
                     'SL SKU của Nhóm Store': len(sku_list_of_group),
                    'SL SKU Thiếu': count_of_missing_sku,
                    'SL SKU Thừa': count_of_redundant_sku,
                    'SKU Thiếu': np.setdiff1d(sku_list_of_group, sku_list_of_store),
                    'SKU Thừa': np.setdiff1d(sku_list_of_store, sku_list_of_group)}])],ignore_index=True,
)


In [106]:
report_df.to_excel('output-files/df_orders_12-4.xlsx',index=False)

In [107]:

portal_order = pd.read_excel('D:\SGC\TPTS_Project\df_portal_order_check\daily_xlsx\DFP_FreshFood_Order_08_04_2024.xlsx')
def clean_df(df):
    col_rename = df.loc[0].to_list()[:8]
    for i, col in enumerate(df.columns):
        if i == 8:
            break
        df.rename(columns={col: col_rename[i]},inplace=True)
    df = df.drop(index=0)
    df_melt = pd.melt(df, id_vars=df.columns[0:8], value_vars=df.columns[8:],
                  var_name = 'Store', value_name='Order_Qty')
    return df_melt
portal_order = clean_df(portal_order)


report_df_portal = pd.DataFrame(columns=['Store', 'SL SKU Đặt', 'Nhóm Store','SL SKU Sai'])


sku_list_each_group = {}
for store_group in dm_cf_store_group['NHOM DS'].unique():
    sku_list_each_group[store_group] = dm_cf_sku[~dm_cf_sku[store_group].isna()]['SKU'].tolist()
sku_list_each_group['No Group'] = pd.read_csv(master_data_dir + '\TPTS_All_Items.csv')['*Item'].tolist()

for store in portal_order['Store'].unique():
    # Get group of store
    group_of_store = dm_cf_store_group.loc[dm_cf_store_group['Mã ST']==store,'NHOM DS'].tolist()
    if not len(group_of_store) == 0:
        group_of_store = group_of_store[0]
    else:
        group_of_store = 'No Group'
    
    # Danh sách SKU của store có trong order
    sku_list_of_store = portal_order[(portal_order['Store']==store)&
                                     (~portal_order['Order_Qty'].isna())]['SKU']
    # Danh sách SKU của nhóm store
    sku_list_of_group = sku_list_each_group[group_of_store]
    # Số lượng SKU có trong order mà không thuộc nhóm store này
    count_of_wrong_sku = len(np.setdiff1d(sku_list_of_store, sku_list_of_group))
    # Danh sach SKU sai
    list_of_wrong_sku = np.setdiff1d(sku_list_of_store, sku_list_of_group)
    report_df_portal = pd.concat([report_df_portal, pd.DataFrame([{'Store': store,
                     'SL SKU Đặt': len(sku_list_of_store),
                     'Nhóm Store': group_of_store,
                     'SL SKU Sai': count_of_wrong_sku,
                     'DS SKU Sai': list_of_wrong_sku}])],ignore_index=True)
    
report_df_portal[report_df_portal['SL SKU Sai']!=0].to_excel('output-files/check-wrong-sku.xlsx')